In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


1. 모든 행정동마다-> 지점이 있는 가장 가까운 행정동 3개에 매핑
2. 은행지점-> 방문하는 행정동들에 대한 시간불편도를 계산 하여 은행개수로 나누기

### 행정동 마다 지점이 있는 행정동3개 매핑

In [ ]:
path="/content/drive/MyDrive/asac/project/eda/행정동_위도경도.xlsx"
loca=pd.read_excel(path)

In [ ]:
loca

In [ ]:
# vpath="/content/drive/MyDrive/asac/project/eda/filtered/v14.xlsx"
# v14=pd.read_excel(vpath)

In [ ]:
vpath="/content/drive/MyDrive/asac/project/eda/filtered/v21.xlsx"
v14=pd.read_excel(vpath)

In [ ]:
v14.replace(to_replace="-",value=0,inplace=True)
v14=v14.loc[:425,:]
v14.tail()

In [ ]:
not_null_pd=v14[v14["신한은행"]!=0]
not_null_pd

In [ ]:
import geopy.distance

In [ ]:
def calc_distance(target):
  distance={}
  latitude=loca[loca["행정동명"]==target]["Latitude"].iat[0]
  longitude=loca[loca["행정동명"]==target]["Longitude"].iat[0]

  for idx, row in not_null_pd.iterrows():
    temp_name=row["행정동명"]

    latitude2=loca[loca["행정동명"]==temp_name]["Latitude"].iat[0]
    longitude2=loca[loca["행정동명"]==temp_name]["Longitude"].iat[0]

    temp_dist=geopy.distance.distance((latitude,longitude),(latitude2,longitude2))

    distance[temp_name]=temp_dist

  return distance



In [ ]:
def get_near_dongs(dong_name):

  x=sorted(calc_distance(dong_name).values())
  sort=sorted(calc_distance(dong_name).items(),key=lambda x:x[1])
  near_dongs=[item[0] for item in sort[:3]]

  return ",".join(near_dongs)

In [ ]:
v14["가까운은행"]=v14.loc[:,"행정동명"].apply(get_near_dongs)

In [ ]:
v14["가까운은행"]

### 매핑된 3개의 지점에 있는 행정동의 수 합하기(= 대체제의 수 구하기)


해당 행정동에 은행지점이 없는 경우 매핑된 3개의 지점이 대체제로 사용, 해당 행정동에 은행지점이 있는 경우 본인제외 3개를 선택하여 대체제로 사용


In [ ]:
v14=pd.read_excel("/content/drive/MyDrive/asac/project/eda/filtered/v22.xlsx")

In [ ]:
x="구로3동,대림2동,구로2동"
x.split(",")

['구로3동', '대림2동', '구로2동']

In [ ]:
v14["가까운은행_대체제용"]=v14["가까운은행"]

In [ ]:
v14["가까운은행_대체제용"]

In [ ]:
def get_near_dongs2(dong_name):
  #해당 행정동에 은행지점이 있는 경우 4개의 행정동 뽑기(본인 제외 3개)
  if not(not_null_pd[not_null_pd["행정동명"]==dong_name].empty):
    x=sorted(calc_distance(dong_name).values())
    sort=sorted(calc_distance(dong_name).items(),key=lambda x:x[1])
    near_dongs=[item[0] for item in sort[1:4]]

  return ",".join(near_dongs)

In [ ]:
v14["가까운은행_대체제용"]=v14[v14["신한은행"]!=0].loc[:,"행정동명"].apply(get_near_dongs2)

In [ ]:
v14["가까운은행_대체제용"]=v14["가까운은행_대체제용"].fillna(v14["가까운은행"])
v14["가까운은행_대체제용"]

0        청운효자동,소공동,충현동
1        가회동,청운효자동,혜화동
2       청운효자동,홍제1동,가회동
3      정릉4동,청운효자동,불광1동
4        사직동,청운효자동,충현동
            ...       
421      광장동,명일1동,성내3동
422     명일1동,성내3동,천호2동
423       고덕2동,명일1동,길동
424       고덕2동,명일1동,길동
425       고덕2동,명일1동,길동
Name: 가까운은행_대체제용, Length: 426, dtype: object

In [ ]:
def count_replacement(x):

  bank_list=x.split(",")
  cnt=0
  for b in bank_list:
    cnt+=not_null_pd[not_null_pd["행정동명"]==b]["유효신한"].iat[0]

  return cnt

In [ ]:
v14["대체제은행수"]=v14.loc[:,"가까운은행_대체제용"].apply(count_replacement)

In [ ]:
v14["대체제은행수"]

0      11
1       3
2       3
3       3
4       4
       ..
421     3
422     3
423     3
424     3
425     3
Name: 대체제은행수, Length: 426, dtype: int64

In [ ]:
v14.to_excel("/content/drive/MyDrive/asac/project/eda/filtered/v22.xlsx")